# Process Data - Download Contributions/Donor Data

This notebook downloads all the contribution/donor data from Open Secrets and creates one combined CSV file. There is no need to run this process because the output file has already been created.

In [40]:
import pandas as pd
import io
import requests

# Import traitor data

In [41]:
traitor_df = pd.read_csv("147_traitors.csv", index_col=False)

In [42]:
traitor_df

,id,full_name,first_name,last_name,state,chamber,top_donors_link
0,N00003028,Robert B. Aderholt,Robert,Aderholt,Ala.,House,https://www.opensecrets.org/members-of-congres...
1,N00030910,Mo Brooks,Mo,Brooks,Ala.,House,https://www.opensecrets.org/members-of-congres...
2,NaN,Jerry Carl,Jerry,Carl,Ala.,House,NaN
3,NaN,Barry Moore,Barry,Moore,Ala.,House,NaN
4,N00035691,Gary Palmer,Gary,Palmer,Ala.,House,https://www.opensecrets.org/members-of-congres...
...,...,...,...,...,...,...,...
142,N00034044,John Kennedy,John,Kennedy,La.,Senate,https://www.opensecrets.org/members-of-congres...
143,N00043298,Cindy Hyde-Smith,Cindy,Hyde-Smith,Miss.,Senate,https://www.opensecrets.org/members-of-congres...
144,N00041620,Josh Hawley,Josh,Hawley,Mo.,Senate,https://www.opensecrets.org/members-of-congres...
145,N00033085,Ted Cruz,Ted,Cruz,Texas,Senate,https://www.opensecrets.org/members-of-congres...


# Remove NAN data

In [43]:
traitor_df = traitor_df.fillna('')

In [44]:
traitor_df

,id,full_name,first_name,last_name,state,chamber,top_donors_link
0,N00003028,Robert B. Aderholt,Robert,Aderholt,Ala.,House,https://www.opensecrets.org/members-of-congres...
1,N00030910,Mo Brooks,Mo,Brooks,Ala.,House,https://www.opensecrets.org/members-of-congres...
2,,Jerry Carl,Jerry,Carl,Ala.,House,
3,,Barry Moore,Barry,Moore,Ala.,House,
4,N00035691,Gary Palmer,Gary,Palmer,Ala.,House,https://www.opensecrets.org/members-of-congres...
...,...,...,...,...,...,...,...
142,N00034044,John Kennedy,John,Kennedy,La.,Senate,https://www.opensecrets.org/members-of-congres...
143,N00043298,Cindy Hyde-Smith,Cindy,Hyde-Smith,Miss.,Senate,https://www.opensecrets.org/members-of-congres...
144,N00041620,Josh Hawley,Josh,Hawley,Mo.,Senate,https://www.opensecrets.org/members-of-congres...
145,N00033085,Ted Cruz,Ted,Cruz,Texas,Senate,https://www.opensecrets.org/members-of-congres...


# Create Master CSV of all contributors 
Download contribution CSV files from Open Secrets for each of the traitors and combine to master CSV - 147_contributors.csv

In [45]:
# starter dataframe for master contributor data
contributor_column_names = ['pacid','orgid','ultorg','total','pacs','indivs','releasedate','traitor_id']
contributor_df = pd.DataFrame(columns = contributor_column_names)

# loop through all traitors
for _, traitor_row in traitor_df.iterrows():     
    if traitor_row.open_secrets_id != '':
        print(f'{traitor_row.full_name} from {traitor_row.state} is a traitor')
        
        # get contributors csv file
        url = f'https://www.opensecrets.org/members-of-congress/{traitor_row.first_name.lower()}-{traitor_row.last_name.lower()}/contributors.csv?cid={traitor_row.open_secrets_id}&cycle=2020&recs=100&type=C'
        data = requests.get(url).content
        df = pd.read_csv(io.StringIO(data.decode('utf-8')))
        df = df.applymap(lambda x: x.strip() if type(x)==str else x)
        
        # remove trivial columns
        df = df.drop(['affiliateid', 'rank', 'client', 'registrant', 'numlobbyists', 'loborgid'], axis=1)
        
        # add traitor data
        df['traitor_id'] = traitor_row.open_secrets_id     
        df['traitor_name'] = traitor_row.full_name
        df['traitor_state'] = traitor_row.state
        df['traitor_chamber'] = traitor_row.chamber
        
        # combine to master contributor file
        frames = [contributor_df, df]
        contributor_df = pd.concat(frames)

# rename columns        
contributor_df = contributor_df.rename(columns={'pacid': 'pac_id', 'orgid': 'id', 'ultorg': 'name', 'indivs': 'individuals', 'releasedate': 'release_date'})  

print('completed')

# save contributors dataframe to csv file
contributor_df.to_csv(r'147_contributors.csv', index=False, header=True)          

Robert B. Aderholt from Ala. is a traitor
Mo Brooks from Ala. is a traitor
Gary Palmer from Ala. is a traitor
Mike Rogers from Ala. is a traitor
Andy Biggs from Ariz. is a traitor
Paul Gosar from Ariz. is a traitor
Debbie Lesko from Ariz. is a traitor
David Schweikert from Ariz. is a traitor
Rick Crawford from Ark. is a traitor
Ken Calvert from Calif. is a traitor
Mike Garcia from Calif. is a traitor
Darrell Issa from Calif. is a traitor
Doug LaMalfa from Calif. is a traitor
Kevin McCarthy from Calif. is a traitor
Devin Nunes from Calif. is a traitor
Doug Lamborn from Colo. is a traitor
Mario Diaz-Balart from Fla. is a traitor
Neal Dunn from Fla. is a traitor
Matt Gaetz from Fla. is a traitor
Brian Mast from Fla. is a traitor
Bill Posey from Fla. is a traitor
John Rutherford from Fla. is a traitor
Greg Steube from Fla. is a traitor
Daniel Webster from Fla. is a traitor
Rick Allen from Ga. is a traitor
Earl L. "Buddy" Carter from Ga. is a traitor
Jody Hice from Ga. is a traitor
Barry Lo

In [ ]:
import requests
for index, row in df.iterrows():  
    if index < 3:
        if row[5] != ' ':
            url = f'https://www.opensecrets.org/members-of-congress/{row[3].lower()}-{row[4].lower()}/contributors.csv?cid={row[5]}&cycle=2020&recs=100&type=C'
            print(url)
            r = requests.get(url)
            with open(str(row[5]) + '.csv', 'wb') as outfile:
                outfile.write(r.content)